# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 22 2022 3:52PM,240251,19,ADV80003536,"AdvaGen Pharma, Ltd",Released
1,Jun 22 2022 3:52PM,240251,19,ADV80003537,"AdvaGen Pharma, Ltd",Released
2,Jun 22 2022 3:52PM,240251,19,ADV80003538,"AdvaGen Pharma, Ltd",Released
3,Jun 22 2022 3:52PM,240251,19,ADV80003539,"AdvaGen Pharma, Ltd",Released
4,Jun 22 2022 3:52PM,240251,19,ADV80003540,"AdvaGen Pharma, Ltd",Released
5,Jun 22 2022 3:52PM,240251,19,ADV80003541,"AdvaGen Pharma, Ltd",Released
6,Jun 22 2022 3:52PM,240251,19,ADV80003542,"AdvaGen Pharma, Ltd",Released
7,Jun 22 2022 3:52PM,240251,19,ADV80003543,"AdvaGen Pharma, Ltd",Released
8,Jun 22 2022 3:52PM,240251,19,ADV80003544,"AdvaGen Pharma, Ltd",Released
9,Jun 22 2022 3:52PM,240251,19,ADV80003545,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,240243,Executing,28
39,240243,Released,8
40,240244,Released,13
41,240249,Released,2
42,240251,Released,17


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240241,NaN,NaN,1.0
240243,NaN,28.0,8.0
240244,NaN,NaN,13.0
240249,NaN,NaN,2.0
240251,NaN,NaN,17.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240135,0.0,0.0,1.0
240137,7.0,24.0,1.0
240147,0.0,0.0,1.0
240161,0.0,0.0,1.0
240163,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240135,0,0,1
240137,7,24,1
240147,0,0,1
240161,0,0,1
240163,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240135,0,0,1
1,240137,7,24,1
2,240147,0,0,1
3,240161,0,0,1
4,240163,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240135,,,1
1,240137,7,24,1
2,240147,,,1
3,240161,,,1
4,240163,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd"
17,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc."
19,Jun 22 2022 3:12PM,240244,10,ISDIN Corporation
32,Jun 22 2022 3:07PM,240243,10,ISDIN Corporation
68,Jun 22 2022 2:59PM,240241,16,Seqirus
69,Jun 22 2022 2:51PM,240239,20,Sartorius Stedim North America
70,Jun 22 2022 2:49PM,240238,12,Hush Hush
71,Jun 22 2022 2:46PM,240237,16,Seqirus
72,Jun 22 2022 2:43PM,240236,10,Eywa Pharma Inc.
89,Jun 22 2022 2:37PM,240235,20,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd",,,17
1,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc.",,,2
2,Jun 22 2022 3:12PM,240244,10,ISDIN Corporation,,,13
3,Jun 22 2022 3:07PM,240243,10,ISDIN Corporation,,28,8
4,Jun 22 2022 2:59PM,240241,16,Seqirus,,,1
5,Jun 22 2022 2:51PM,240239,20,Sartorius Stedim North America,,,1
6,Jun 22 2022 2:49PM,240238,12,Hush Hush,,1,
7,Jun 22 2022 2:46PM,240237,16,Seqirus,,,1
8,Jun 22 2022 2:43PM,240236,10,Eywa Pharma Inc.,,10,7
9,Jun 22 2022 2:37PM,240235,20,Sartorius Stedim North America,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd",17,,
1,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc.",2,,
2,Jun 22 2022 3:12PM,240244,10,ISDIN Corporation,13,,
3,Jun 22 2022 3:07PM,240243,10,ISDIN Corporation,8,28,
4,Jun 22 2022 2:59PM,240241,16,Seqirus,1,,
5,Jun 22 2022 2:51PM,240239,20,Sartorius Stedim North America,1,,
6,Jun 22 2022 2:49PM,240238,12,Hush Hush,,1,
7,Jun 22 2022 2:46PM,240237,16,Seqirus,1,,
8,Jun 22 2022 2:43PM,240236,10,Eywa Pharma Inc.,7,10,
9,Jun 22 2022 2:37PM,240235,20,Sartorius Stedim North America,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd",17,,
1,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc.",2,,
2,Jun 22 2022 3:12PM,240244,10,ISDIN Corporation,13,,
3,Jun 22 2022 3:07PM,240243,10,ISDIN Corporation,8,28,
4,Jun 22 2022 2:59PM,240241,16,Seqirus,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd",17.0,NaN,NaN
1,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc.",2.0,NaN,NaN
2,Jun 22 2022 3:12PM,240244,10,ISDIN Corporation,13.0,NaN,NaN
3,Jun 22 2022 3:07PM,240243,10,ISDIN Corporation,8.0,28.0,NaN
4,Jun 22 2022 2:59PM,240241,16,Seqirus,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd",17.0,0.0,0.0
1,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc.",2.0,0.0,0.0
2,Jun 22 2022 3:12PM,240244,10,ISDIN Corporation,13.0,0.0,0.0
3,Jun 22 2022 3:07PM,240243,10,ISDIN Corporation,8.0,28.0,0.0
4,Jun 22 2022 2:59PM,240241,16,Seqirus,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1921642,37.0,40.0,0.0
12,240238,0.0,1.0,0.0
15,960732,3.0,26.0,7.0
16,960875,3.0,1.0,0.0
19,1681443,27.0,7.0,0.0
20,1201101,21.0,24.0,1.0
21,480408,2.0,0.0,0.0
22,240218,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1921642,37.0,40.0,0.0
1,12,240238,0.0,1.0,0.0
2,15,960732,3.0,26.0,7.0
3,16,960875,3.0,1.0,0.0
4,19,1681443,27.0,7.0,0.0
5,20,1201101,21.0,24.0,1.0
6,21,480408,2.0,0.0,0.0
7,22,240218,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,37.0,40.0,0.0
1,12,0.0,1.0,0.0
2,15,3.0,26.0,7.0
3,16,3.0,1.0,0.0
4,19,27.0,7.0,0.0
5,20,21.0,24.0,1.0
6,21,2.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,37.0
1,12,Released,0.0
2,15,Released,3.0
3,16,Released,3.0
4,19,Released,27.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,7.0,0.0,0.0,1.0,0.0,0.0
Executing,40.0,1.0,26.0,1.0,7.0,24.0,0.0,0.0
Released,37.0,0.0,3.0,3.0,27.0,21.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,7.0,0.0,0.0,1.0,0.0,0.0
1,Executing,40.0,1.0,26.0,1.0,7.0,24.0,0.0,0.0
2,Released,37.0,0.0,3.0,3.0,27.0,21.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,7.0,0.0,0.0,1.0,0.0,0.0
1,Executing,40.0,1.0,26.0,1.0,7.0,24.0,0.0,0.0
2,Released,37.0,0.0,3.0,3.0,27.0,21.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()